In [ ]:
from bs4 import BeautifulSoup
import sys

def logL(x):
    
    #print(seq1[10],seq2[10])
    LogL = 0
    #len(seq1) -1
    for i in range(1,100): 
        
        #Tree structure, this should come from a .nex file
        N = 20
        #Sample size per loci
        n = 100
        T = Tree(np.array([[1,2],[3,4],[5,6]]),np.array([5,5,6,7,6,7]),x)
        #T = Tree(np.array([[1,2]]),np.array([3,3]),x)
        #Given frequency
        C = np.array([int(seq1[i]),int(seq2[i]),int(seq3[i]),int(seq4[i])])
        #Get Chebychev Coefficients of initial condition
        CC = np.zeros([len(C),N+1])
        for i in range(0,len(C)):
            CC[i] = Chebychev_transform2(n_sample,C[i],N) 
        #selection and mutation (multiply u,v by 2)    
        a = np.array([[0.006,-0.012,0,0],[0.005,-0.01,0,0]])
        #genetic drift
        b = np.array([0,0.5,-0.5,0])
        LogL += Tree_integrator(T,C,N,n,CC,a,b)
    return LogL


def BranchSupp(x):
	return all(0 < x )&all(x < 20)
###          we define the objective function with the U function
###          which is -log of the density function.
###          The support is defined in a separate function.
###   The dimension of the parameter space is n

'''Get the SimSnap simulations into format '''
handler = open("testSimSnap_tree_1.xml","r" )
contents = handler.read()
soup = BeautifulSoup(contents,'xml')
sequence = soup.find_all('sequence')
seq1 = sequence[0].text
seq2 = sequence[1].text
seq3 = sequence[2].text
seq4 = sequence[3].text
seq1 = seq1.split(",")
seq2 = seq2.split(",")
seq3 = seq3.split(",")
seq4 = seq4.split(",")

Tree_Inference = pytwalk( n=6, U=logL, Supp=BranchSupp)

#### This would run the twalk

Tree_Inference.Run( T=10000, x0=1*np.ones(6), xp0=2*np.ones(6))

Tree_Inference.Save("Tree_Inference_twalk1.dat")

#Gets output into Tracer format
x = Tree_Inference.Output
n = np.shape(x)
z = np.zeros([int(n[0])+1, int(n[1])+1])
z[1:,1:] = x
y = np.arange(0,n[0] +1)
z[:,0] = y
header = np.arange(1,n[1]+1)
header_str = str(np.arange(1,n[1]+1))
header_str = header_str[1:-1]
header_str = header_str.replace(" ", "\t")
np.savetxt( 'test.log', z, delimiter='\t', header='state' + header_str )

#Tree_Inference.Ana
#Tree_Inference.Hist( par=0 )

#plt.show()
#Tree_Inference.Hist( par=1 )
#plt.show()
#Tree_Inference.Hist( par=2 )
#plt.show()
#Tree_Inference.Hist( par=3 )
#plt.show()
#Tree_Inference.Hist( par=4 )
#plt.show()
#Tree_Inference.Hist( par=5 )
#plt.show()


In [7]:
%matplotlib inline
import scipy.integrate as integrate
import scipy as sp
import scipy.stats as sps
import numpy as np
import numpy.polynomial.chebyshev as ch
from numpy.linalg import inv
from scipy.linalg import expm
import matplotlib.pyplot as plt
import numpy.polynomial.chebyshev as ch
from pytwalk import * 

class Tree:
    def __init__(self,structure_pc, structure_cp, time):
        self.structure_pc = structure_pc 
        self.structure_cp = structure_cp
        self.time = time
        
def x_times_Tx(N): #Transpose of Operator B defined as x*f(x) (Gordon's thesis Propositon.5 page. 78)
    B = sp.sparse.diags([0.25, 0.5, 0.25], [-1, 0, 1], shape=(N, N)).toarray()
    B[1,0] = 0.5
    return B

def Chebychev_transform2(n,m,N):
    circle = np.arange(0,np.pi+1/n,1/n)
    x = (np.cos(circle) + 1)/2
    f = sps.binom.pmf(m, n, x, loc=0)
    #specify Chebychev bases
    chebexp_shift = ch.Chebyshev.fit(x,f,N,domain=[0,1])
    return chebexp_shift.coef

def Dd(N): #Derivative operator
    D = np.zeros([N,N])
    D[0,np.arange(1,N,2)] = 2*np.arange(1,N,2)
    for i in range(0,N-2):
        D[i+1,np.arange(i+2,N,2)] = 4*(np.arange(i+1,N-1,2)+1)
    return D

def Tree_integrator(T,C,N,n_sample,CC,a,b): 
    nnodes = np.shape(T.structure_cp)[0] + 1
    n = (nnodes+1.)/2.
    F = np.zeros([N,nnodes])
    logL = 0
    circle = np.arange(0,np.pi+1/n_sample,1/n_sample)
    p = (np.cos(circle) + 1)/2
    #4.5 Proposition 5 Operator B
    S = x_times_Tx(N)
    S2 = np.dot(S,S)
    S3 = np.dot(S2,S)
    #First Derivative       
    D = Dd(N)
    #Second Derivative
    D2 = np.dot(D,D)
    #Q-matrix (although it's called M)
    M = (a[0,0]*D) + (a[0,1]*np.dot(S,D)) + (a[0,2]*np.dot(S2,D)) + (a[0,3]*np.dot(S3,D)) + .5*((b[0]*D2) + (b[1]*np.dot(S,D2)) + (b[2]*np.dot(S2,D2))) 
#---step1-a---    
    j=0
    for i in range(0,int(n)): 
        tt = T.time[i] #branch length
        #these points have to be consistent with the number of points
        cc = CC[i,:]   
        j += 1
#---step1-b,c---
        F[:,i] = np.transpose(np.dot(expm(M*tt),cc[:-1]))
#---step2---  
    #Adhoc for now
    M = (a[1,0]*D) + (a[1,1]*np.dot(S,D)) + (a[1,2]*np.dot(S2,D)) + (a[1,3]*np.dot(S3,D)) + .5*((b[0]*D2) + (b[1]*np.dot(S,D2)) + (b[2]*np.dot(S2,D2))) 
    for i in range(int(n),nnodes-1):
#---step2-a,b,c---
        u = T.structure_pc[i - int(n),0]
        v = T.structure_pc[i - int(n),1]
        F1 = F[:,u-1]
        F2 = F[:,v-1]
        tt = T.time[i]
#---step2d---
#   4.3.3  Proposition 1
        F1chebpts = ch.chebval(2*p-1,  F1)
        F2chebpts = ch.chebval(2*p-1,  F2)
        Ffit = ch.Chebyshev.fit(p,F1chebpts*F2chebpts,N,domain=[0,1])
        cc = Ffit.coef            
#---step2-e,f---
        F[:,i] = np.transpose((np.dot(expm(M*tt),cc[:-1])))
#---step3---    
    r = nnodes - 1
    u = T.structure_pc[r - int(n),0]
    v = T.structure_pc[r - int(n),1]
#---step4,5---
    F1 = F[:,u-1]
    F2 = F[:,v-1]
#---step6--- #The root branch has zero length    
    f = lambda p: ch.chebval(2*p-1, F1)*ch.chebval(2*p-1, F2)*sps.beta.pdf(p,0.006,0.006)
    result = integrate.quad(f, 0, 1)
#---step7---    
    like = result[0]
    #print(like)
    if 1>=like>=0:
        logL = np.log(like)
    else:
        logL = -10000   
    return -logL

#Tree structure, this should come from a .nex file
T = Tree(np.array([[1,2],[3,4],[5,6]]),np.array([5,5,6,7,6,7]),np.array([0.1,0.1,0.1,0.1,0.1,0.1]))
#T = Tree(np.array([[1,2]]),np.array([3,3]),np.array([0.1,1]))
#Sample size
n = 100
# Number of red alleles
C = np.array([60,70, 65,50])
#Chebychev basis
N = 20
#Get Chebychev Coefficients of initial condition
CC = np.zeros([len(C),N+1])
for i in range(0,len(C)):
    CC[i] = Chebychev_transform2(n,C[i],N)  
#selection and mutation    
a = np.array([[.5,-1,0,0],[.4,-0.5,0,0]])
#genetic drift
b = np.array([0,0.5,-0.5,0])
#Return -likelihood
Tree_integrator(T,C,N,n,CC,a,b)

20.759584190902416

In [ ]:
help(ch.Chebyshev.fit)